In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import scrapy
import numpy as np
import os
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

### define headers to request webpage as a human browser

In [2]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

### here we define:
+ page: the address to scrape from
+ pageTree: use requests to grab the code and assign to pageTree
+ pageSoup: grab code content and parse into html for us to finally scrape from

In [4]:
page = 'https://www.transfermarkt.co.uk/transfers/transferrekorde/statistik/top/plus/0/galerie/0?saison_id=2000'
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

In [5]:
pageSoup

\n<!DOCTYPE html>\n\n<!-- paulirish.com/2008/conditional-stylesheets-vs-css-hacks-answer-neither/ -->\n<!--[if IE 7]>\n<html class="ie7 oldie" lang="en"> <![endif]-->\n<!--[if IE 8]>\n<html class="no-js lt-ie9" lang="en"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html class="no-js" lang="en"> <!--<![endif]-->\n<head>\n<meta charset="unicode-escape"/>\n<meta content="IE=edge" http-equiv="x-ua-compatible"><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newr

### we can now scrap transfer values data

### the first parameter is tag, second is class - both can be found in the page source

In [6]:
Players = pageSoup.find_all('a'
                            , {'class':'spielprofil_tooltip'}
                           )
Values = pageSoup.find_all('td'
                           , {'class': 'rechts hauptlink'}
                          )

### now we can put them in a nice dataframe - in this case of players and their transfer values

In [11]:
len(Values)

25

In [12]:
df_tf_values = pd.DataFrame()
for i, value in enumerate(Values):
    df_tf_values = df_tf_values.append({'Player': Players[i].text
                                        , 'Value': value.text
                                       }
                                       , ignore_index=True
                                       
                                      )

In [13]:
df_tf_values

Player    Value
0                 Luís Figo  £54.00m
1             Hernán Crespo  £51.13m
2             Marc Overmars  £36.00m
3         Gabriel Batistuta  £32.54m
4            Nicolas Anelka  £31.05m
5             Rio Ferdinand  £23.40m
6          Flávio Conceicao  £22.50m
7            Savo Milosevic  £22.50m
8           David Trézéguet  £20.92m
9             Claudio López  £20.70m
10  Jimmy Floyd Hasselbaink  £20.25m
11             Gerard López  £19.44m
12                    Lucas  £19.17m
13              Pablo Aimar  £19.13m
14            Wálter Samuel  £18.72m
15            Shabani Nonda  £18.00m
16             Robbie Keane  £17.55m
17                José Mari  £17.10m
18          Jonathan Zebina  £16.56m
19                  Émerson  £16.20m
20           Tore André Flo  £16.20m
21            Serhii Rebrov  £16.20m
22           Angelo Peruzzi  £16.11m
23            Diego Tristán  £15.98m
24          Sylvain Wiltord  £15.75m

In [14]:
for i, player in enumerate(Players):
    print(i, player.text)

(0, u'Lu\xeds Figo')
(1, u'Hern\xe1n Crespo')
(2, u'Marc Overmars')
(3, u'Gabriel Batistuta')
(4, u'Nicolas Anelka')
(5, u'Rio Ferdinand')
(6, u'Fl\xe1vio Conceicao')
(7, u'Savo Milosevic')
(8, u'David Tr\xe9z\xe9guet')
(9, u'Claudio L\xf3pez')
(10, u'Jimmy Floyd Hasselbaink')
(11, u'Gerard L\xf3pez')
(12, u'Lucas')
(13, u'Pablo Aimar')
(14, u'W\xe1lter Samuel')
(15, u'Shabani Nonda')
(16, u'Robbie Keane')
(17, u'Jos\xe9 Mari')
(18, u'Jonathan Zebina')
(19, u'\xc9merson')
(20, u'Tore Andr\xe9 Flo')
(21, u'Serhii Rebrov')
(22, u'Angelo Peruzzi')
(23, u'Diego Trist\xe1n')
(24, u'Sylvain Wiltord')
(25, u'M. Eggestein')
(26, u'Ui-jo Hwang')
(27, u'\xc9. Milit\xe3o')
(28, u'S. Gudbjargarson')
(29, u'S. Gudbjargarson')


### for some reason, Players has more entries than Values

### it's right, the last 3 entries in Players are gibberish

## Now, let's try scraping transfer values data from a more recent season, and try to loop over all the search results pages

In [ ]:
page = 'https://www.transfermarkt.co.uk/transfers/transferrekorde/statistik?saison_id=2018

### apparently we can add '&page=n' where n is the index of search page and we're good

### also we can change which season we're looking at by edit the parameter 'saison_id'

In [8]:
''.join(['hello','world'])

'helloworld'

In [12]:
process_start = datetime.now()

df_TransferValues = pd.DataFrame()
seasons = np.arange(2008,2019)
search_number = np.arange(1,11)
for season in seasons:
    print('Loading season', '-'.join([str(season), str(season+1)]))
    for i in range(1,11):
        print('Loading page',i)
        page_core = 'https://www.transfermarkt.co.uk/transfers/transferrekorde/statistik?'
        page_season = 'saison_id=' + str(season)
        page_number = '&page=' + str(i)

        page = ''.join([page_core, page_season, page_number])
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all('a'
                                    , {'class':'spielprofil_tooltip'}
                                   )
        Values = pageSoup.find_all('td'
                                   , {'class': 'rechts hauptlink'}
                                  )
        for i, value in enumerate(Values):
            player = Players[i].text
            value = value.text
            
            df_TransferValues = df_TransferValues.append({'Player':player
                                                         , 'Value': value
                                                         , 'Season': season
                                                         }
                                                         , ignore_index=True
                                                        )

print('Time elapsed:', datetime.now() - process_start)
df_TransferValues.head()

('Loading season', '2008-2009')
('Loading page', 1)
('Loading page', 2)
('Loading page', 3)
('Loading page', 4)
('Loading page', 5)
('Loading page', 6)
('Loading page', 7)
('Loading page', 8)
('Loading page', 9)
('Loading page', 10)
('Loading season', '2009-2010')
('Loading page', 1)
('Loading page', 2)
('Loading page', 3)
('Loading page', 4)
('Loading page', 5)
('Loading page', 6)
('Loading page', 7)
('Loading page', 8)
('Loading page', 9)
('Loading page', 10)
('Loading season', '2010-2011')
('Loading page', 1)
('Loading page', 2)
('Loading page', 3)
('Loading page', 4)
('Loading page', 5)
('Loading page', 6)
('Loading page', 7)
('Loading page', 8)
('Loading page', 9)
('Loading page', 10)
('Loading season', '2011-2012')
('Loading page', 1)
('Loading page', 2)
('Loading page', 3)
('Loading page', 4)
('Loading page', 5)
('Loading page', 6)
('Loading page', 7)
('Loading page', 8)
('Loading page', 9)
('Loading page', 10)
('Loading season', '2012-2013')
('Loading page', 1)
('Loading page',

Player  Season    Value
0              Robinho  2008.0  £38.70m
1     Dimitar Berbatov  2008.0  £34.20m
2           Dani Alves  2008.0  £31.95m
3                Danny  2008.0  £27.00m
4  Klaas-Jan Huntelaar  2008.0  £24.30m

### We've successfully downloaded transfer values of players from the last 10 seasons. But there are in fact more information we can get e.g. age, market value at the time of transfer, where was the player transfered from and to, position, nationality etc. Let's see if we can crawl that data as well

In [34]:
page_number

'&page=10'

In [168]:
page = ''.join([page_core, page_season, page_number.replace('0','')])
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
#pageSoup = BeautifulSoup(pageTree.content, 'lxml')

In [169]:
Players = pageSoup.find_all('a'
                            , {'class':'spielprofil_tooltip'}
                           )
Fees = pageSoup.find_all('td'
                           , {'class': 'rechts hauptlink'}
                          )
Values = pageSoup.find_all('td'
                    , {'class':'rechts'}
                   )
Destination = pageSoup.find_all('a', {'class':'vereinprofil_tooltip'})

In [173]:
len(Values)

25

### so far we managed to obtain player destination. As for the departure clubs or player value at the time of transfer, somehow they elude me still :/ . We'll come back later, perhaps with Scrapy

### for now, we're content with fees and destination clubs. Let's modify the code slightly

In [13]:
process_start = datetime.now()

df_TransferValues = pd.DataFrame()
seasons = np.arange(2008,2019)
search_number = np.arange(1,21)
for season in seasons:
    current_season = '-'.join([str(season), str(season+1)])
    print('Loading season', '-'.join([str(season), str(season+1)]))
    for i in range(1,11):
        #print('Loading page',i)
        page_core = 'https://www.transfermarkt.co.uk/transfers/transferrekorde/statistik?'
        page_season = 'saison_id=' + str(season)
        page_number = '&page=' + str(i)

        page = ''.join([page_core, page_season, page_number])
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all('a'
                                    , {'class':'spielprofil_tooltip'}
                                   )
        Fees = pageSoup.find_all('td'
                                   , {'class': 'rechts hauptlink'}
                                  )
        Values = pageSoup.find_all('td'
                    , {'class':'rechts'}
                   )
        Destination = pageSoup.find_all('a', {'class':'vereinprofil_tooltip'})
        
        for i, fee in enumerate(Fees):
            player = Players[i].text
            fee = fee.text
            value = Values[i].text
            destination = Destination[2*i+1].text
            if (destination == ''):
                #print(destination)
                print( 'Player ' + player + ' has no destination in page ' + str(i) + '??')
            
            df_TransferValues = df_TransferValues.append({'Player': player
                                                         , 'TransferFees': fee
                                                         #, 'Transfer Value': value
                                                         , 'Destination': destination
                                                         , 'Season': current_season
                                                         }
                                                         , ignore_index=True
                                                        )

print('Time elapsed:', datetime.now() - process_start)
df_TransferValues.head()

Loading season 2008-2009
Loading season 2009-2010
Loading season 2010-2011
Loading season 2011-2012
Loading season 2012-2013
Loading season 2013-2014
Loading season 2014-2015
Loading season 2015-2016
Loading season 2016-2017
Loading season 2017-2018
Loading season 2018-2019
Time elapsed: 0:03:01.824753


Destination               Player     Season TransferFees
0      Man City              Robinho  2008-2009      £38.70m
1       Man Utd     Dimitar Berbatov  2008-2009      £34.20m
2  FC Barcelona           Dani Alves  2008-2009      £31.95m
3    Zenit S-Pb                Danny  2008-2009      £27.00m
4   Real Madrid  Klaas-Jan Huntelaar  2008-2009      £24.30m

### We have successfully extracted the data. Still we have to perform some preprocessing first before any analysis

In [16]:
df_TransferValues.shape

(2750, 4)

#### We note that the column fees and values are still strings, we'll convert these into numbers. 

#### I figure the fees are in format £...m so I would extract the number value accordingly. I would've used regular expression but memory doesn't serve me well on that at the moment.

In [5]:
def get_money_value(value):
    value = value.split('£')[-1]
    value = value[:-1].split('.')
    value = np.int(value[0]) + np.int(value[1])/100.
    value = value * 10 ** 6
    return value

In [15]:
df_TransferValues[['Loan' in fee for fee in df_TransferValues['TransferFees'].values]]

Destination                Player     Season      TransferFees
396           Dnipro         Nery Castillo  2009-2010   Loan fee:£4.50m
608         AC Milan    Zlatan Ibrahimovic  2010-2011   Loan fee:£5.40m
636       SSC Napoli        Edinson Cavani  2010-2011   Loan fee:£4.50m
658         Juventus    Fabio Quagliarella  2010-2011   Loan fee:£4.05m
682      Real Madrid     Emmanuel Adebayor  2010-2011   Loan fee:£3.60m
748         AC Milan  Kevin-Prince Boateng  2010-2011   Loan fee:£2.70m
864        Al-Ain FC          Asamoah Gyan  2011-2012   Loan fee:£6.30m
1052         AS Roma         Mattia Destro  2012-2013  Loan fee:£10.35m
1151       Liverpool            Nuri Sahin  2012-2013   Loan fee:£4.50m
1615         Man Utd                Falcao  2014-2015   Loan fee:£6.84m
1637   VfL Wolfsburg     Josuha Guilavogui  2014-2015   Loan fee:£6.12m
1720         AS Roma            Salih Ucan  2014-2015   Loan fee:£4.28m
1942         Chelsea                Falcao  2015-2016   Loan fee:£6.30m
1955  Bayern Munich         Kingsley Coman  2015-2016   Loan fee:£6.30m
2264        Paris SG         Kylian Mbappé  2017-2018  Loan fee:£40.50m
2374  Bayern Munich        James Rodríguez  2017-2018  Loan fee:£11.70m
2454         Swansea        Renato Sanches  2017-2018   Loan fee:£7.65m
2457        AC Milan         Franck Kessié  2017-2018   Loan fee:£7.20m
2478      Stoke City            Kurt Zouma  2017-2018   Loan fee:£7.02m
2488           Lazio            Pedro Neto  2017-2018   Loan fee:£6.75m
2614        Atalanta          Duván Zapata  2018-2019  Loan fee:£12.60m
2688        AC Milan       Gonzalo Higuaín  2018-2019   Loan fee:£8.10m
2689         Chelsea       Gonzalo Higuaín  2018-2019   Loan fee:£8.10m
2737         Everton            Kurt Zouma  2018-2019   Loan fee:£7.11m

In [18]:
df_TransferValues[['loan' in fee for fee in df_TransferValues['TransferFees'].values]]

Destination    Player     Season TransferFees
1619  Real Madrid  Casemiro  2014-2015  End of loan

In [7]:
#df_TransferValues['Transfer Fees'] = df_TransferValues['Transfer Fees'].apply(get_money_value)
for i, value in enumerate(df_TransferValues['Transfer Fees'].values):
    value = get_money_value(value)

ValueError: invalid literal for int() with base 10: 'End of loa'

### Now that the data has been processed. We can start the preliminary analysis

In [15]:
df_TransferValues.head()

Player  Season    Value
0              Robinho  2008.0  £38.70m
1     Dimitar Berbatov  2008.0  £34.20m
2           Dani Alves  2008.0  £31.95m
3                Danny  2008.0  £27.00m
4  Klaas-Jan Huntelaar  2008.0  £24.30m

In [13]:
df_TransferValues.shape

(2750, 3)

In [14]:
df_TransferValues.Season.value_counts()

2015.0    250
2018.0    250
2014.0    250
2017.0    250
2013.0    250
2016.0    250
2012.0    250
2011.0    250
2010.0    250
2009.0    250
2008.0    250
Name: Season, dtype: int64

In [20]:
sns.distplot(df_TransferValues.Value, kde=False)

TypeError: unsupported operand type(s) for /: 'unicode' and 'long'